In [1]:
import os
import sys
import timeit
import numpy as np
import theano
import theano.tensor as T
import lasagne
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d
from logistic_sgd import load_data
from lasagne.layers import get_output, InputLayer, DenseLayer, Upscale2DLayer, ReshapeLayer
from lasagne.regularization import regularize_network_params, l2, l1
import gzip
import pickle
import time

Using gpu device 0: TITAN X (Pascal) (CNMeM is disabled, cuDNN 5105)


In [2]:
batch_size = 1000
f = gzip.open('/home/rui/Downloads/mnist.pkl.gz', 'rb')
try:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
except:
    train_set, valid_set, test_set = pickle.load(f)
f.close()
X_train, y_train = train_set
y_train = np.asarray(y_train, dtype = np.int32)
X_test, y_test = test_set
y_test = np.asarray(y_test, dtype = np.int32)

In [3]:
X_train.shape

(50000, 784)

In [4]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]


# MLP with 2 fully connected hidden layers, each layer has 1024 hidden units. 

In [5]:
def build_model(input_var, hidden_neurons=1024):
    l_in = InputLayer(shape=(None, 784), input_var=input_var)
    l_hidden1 = DenseLayer(l_in, num_units=hidden_neurons, W=lasagne.init.HeNormal(gain='relu'))
    l_hidden2 = DenseLayer(l_hidden1, num_units=hidden_neurons, W=lasagne.init.HeNormal(gain='relu'))
    l_out = DenseLayer(lasagne.layers.DropoutLayer(l_hidden2), num_units=10, nonlinearity=lasagne.nonlinearities.softmax, W=lasagne.init.HeNormal(gain='relu'))
    return l_out

# Train with adagrad algorithm, weight decay with parameter 1e-4

## final accuracy: 97.77%

In [6]:
num_epochs = 500
input_var = T.matrix('inputs')
target_var = T.ivector('targets')
network = build_model(input_var, 1024)
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
l2_penalty = regularize_network_params(network, l2)
loss = loss.mean() + 1e-4*l2_penalty
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.adagrad(loss, params, learning_rate=0.1)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                    dtype=theano.config.floatX)
train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1
        # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))

test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
final_acc = test_acc / test_batches
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    final_acc * 100))


Epoch 1 of 200 took 0.876s
  training loss:		20.089788
Epoch 2 of 200 took 0.818s
  training loss:		1.018199
Epoch 3 of 200 took 0.815s
  training loss:		0.859098
Epoch 4 of 200 took 0.816s
  training loss:		0.774754
Epoch 5 of 200 took 0.816s
  training loss:		0.715267
Epoch 6 of 200 took 0.814s
  training loss:		0.671893
Epoch 7 of 200 took 0.821s
  training loss:		0.635314
Epoch 8 of 200 took 0.819s
  training loss:		0.604176
Epoch 9 of 200 took 0.817s
  training loss:		0.575619
Epoch 10 of 200 took 0.815s
  training loss:		0.551321
Epoch 11 of 200 took 0.816s
  training loss:		0.529764
Epoch 12 of 200 took 0.815s
  training loss:		0.510364
Epoch 13 of 200 took 0.816s
  training loss:		0.493556
Epoch 14 of 200 took 0.814s
  training loss:		0.479244
Epoch 15 of 200 took 0.814s
  training loss:		0.462639
Epoch 16 of 200 took 0.820s
  training loss:		0.450371
Epoch 17 of 200 took 0.832s
  training loss:		0.436246
Epoch 18 of 200 took 0.814s
  training loss:		0.425166
Epoch 19 of 200 to

# add bottle neck(128 units) between input layer and hidden1, hidden1 and hidden2. 

## final accuracy: 96.74%

In [7]:
def build_NIN_model(input_var, hidden_neurons=1024, bottle_neck=128):
    l_in = InputLayer(shape=(None, 784), input_var=input_var)
    l_b1 = DenseLayer(l_in, num_units=bottle_neck, W=lasagne.init.HeNormal(gain='relu'))
    l_hidden1 = DenseLayer(l_b1, num_units=hidden_neurons, W=lasagne.init.HeNormal(gain='relu'))
    l_b2 = DenseLayer(l_hidden1, num_units=bottle_neck, W=lasagne.init.HeNormal(gain='relu'))
    l_hidden2 = DenseLayer(l_b2, num_units=hidden_neurons, W=lasagne.init.HeNormal(gain='relu'))
    l_out = DenseLayer(lasagne.layers.DropoutLayer(l_hidden2), num_units=10, nonlinearity=lasagne.nonlinearities.softmax, W=lasagne.init.HeNormal(gain='relu'))
    return l_out

In [8]:
num_epochs = 500
input_var = T.matrix('inputs')
target_var = T.ivector('targets')
network = build_NIN_model(input_var, 1024, 128)
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
l2_penalty = regularize_network_params(network, l2)
loss = loss.mean() + 1e-4*l2_penalty
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.adagrad(loss, params, learning_rate=0.05)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                    dtype=theano.config.floatX)
train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1
        # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))

test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
final_acc = test_acc / test_batches
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    final_acc * 100))

Epoch 1 of 200 took 0.591s
  training loss:		7.346979
Epoch 2 of 200 took 0.586s
  training loss:		0.731895
Epoch 3 of 200 took 0.602s
  training loss:		0.554173
Epoch 4 of 200 took 0.575s
  training loss:		0.477607
Epoch 5 of 200 took 0.574s
  training loss:		0.432261
Epoch 6 of 200 took 0.577s
  training loss:		0.395385
Epoch 7 of 200 took 0.579s
  training loss:		0.372792
Epoch 8 of 200 took 0.574s
  training loss:		0.352455
Epoch 9 of 200 took 0.574s
  training loss:		0.334723
Epoch 10 of 200 took 0.574s
  training loss:		0.319969
Epoch 11 of 200 took 0.575s
  training loss:		0.306710
Epoch 12 of 200 took 0.643s
  training loss:		0.296258
Epoch 13 of 200 took 0.577s
  training loss:		0.285162
Epoch 14 of 200 took 0.576s
  training loss:		0.278656
Epoch 15 of 200 took 0.578s
  training loss:		0.267407
Epoch 16 of 200 took 0.577s
  training loss:		0.258304
Epoch 17 of 200 took 0.575s
  training loss:		0.252724
Epoch 18 of 200 took 0.576s
  training loss:		0.246525
Epoch 19 of 200 too